In [14]:
# Import Packages
from google.cloud import bigquery
import pandas as pd    
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from datetime import datetime 
import time  
import os

In [2]:
# Initialize Client Object
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'adrianjuliusaluoch.json'
client = bigquery.Client(project='project-adrian-julius-aluoch')

#### **1. Food Cupboard**

In [ ]:
# Set up Selenium Firefox WebDriver
options = Options()
options.headless = True  # Run Firefox in headless mode, without opening a browser window
driver = webdriver.Firefox(options=options)

# Define Links
links = [
        'food-additives',
        'snacks',
        'fats-oils',
        'breakfast',
        'commodities',
        'canned-frozen-meals',
        'naivas-dry-cereals-nuts'
    ]

# Define Loop
for link in links:
    # Specify Website URL
    url = 'https://naivas.online/' + str(link)

    # Load Webpage
    driver.get(url)

    # Wait for Page to fully render
    driver.implicitly_wait(4)

    # Get Initial Page Height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll Down to Page Bottom
    while True:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  # Wait 4 seconds for page to load more content

        # Compare Final Scroll Height with Initial Scroll Height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # If Height is constant, exit loop
        last_height = new_height

        # Limit Number of Listings
        if len(BeautifulSoup(driver.page_source, 'html.parser').find_all('div', class_='h-full')) >= 500:
            break

    # Get Page Source after JavaScript Rendering
    page_source = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'lxml')
    groceries = soup.find_all('div', class_='h-full')

    # Extract grocery Listings from Jiji
    bigdata = pd.DataFrame()
    for grocery in groceries:
        try:
            product_name = grocery.find('span',class_='line-clamp-2 text-ellipsis').text.strip()
            product_category = 'Food Cupboard'
            product_subcategory = link
            try:
                product_availability = grocery.find('p',class_='text-naivas-red text-sm font-bold').text.strip()
            except Exception as e:
                product_availability = np.nan
            try:
                final_price = grocery.find('span',class_='font-bold text-naivas-green mb-1 md:mb-0 pe-2').text.strip()
            except Exception as e:
                final_price = np.nan
            last_scraped = datetime.now()
            try:
                initial_price = grocery.find('span',class_='text-red-600 text-xs line-through font-light').text.strip()
            except Exception as e:
                initial_price = np.nan

            # Append Data to DataFrame
            data = pd.DataFrame({
                    'product_name':[product_name],
                    'product_category':[product_category],
                    'product_subcategory':[product_subcategory],
                    'product_availability':[product_availability],
                    'initial_price':[initial_price],
                    'final_price':[final_price],
                    'last_scraped':[last_scraped]
                })

            bigdata = pd.concat([bigdata,data])

        except Exception as e:
            pass

    # Handle Database Import Error
    bigdata.to_csv('naivas_metadata.csv',index=False)
    bigdata = pd.read_csv('naivas_metadata.csv') 
    bigdata['last_scraped'] = pd.to_datetime(bigdata['last_scraped'])


    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(bigdata, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
    sql = (
           'SELECT *'
           'FROM `cronjobs.naivas`'
           )
    
    # Run SQL Query
    data = client.query(sql).to_dataframe()

    # Delete Original Table
    client.delete_table(table_id)
    print(f"Table deleted successfully.")

    # Check Total Number of Duplicate Records
    duplicated = data.duplicated(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price']).sum()
    
    # Remove Duplicate Records
    data.drop_duplicates(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price'],inplace=True)

    # Define the dataset ID and table ID
    dataset_id = 'cronjobs'
    table_id = 'naivas'
    
    # Define the table schema
    schema = [
        bigquery.SchemaField("product_name", "STRING"),
        bigquery.SchemaField("product_category", "STRING"),
        bigquery.SchemaField("product_subcategory", "STRING"),
        bigquery.SchemaField("product_availability", "STRING"),
        bigquery.SchemaField("initial_price", "STRING"),
        bigquery.SchemaField("final_price", "STRING"),
        bigquery.SchemaField("last_scraped", "TIMESTAMP")
    ]
    
    # Define the table reference
    table_ref = client.dataset(dataset_id).table(table_id)
    
    # Create the table object
    table = bigquery.Table(table_ref, schema=schema)
    
    # Create the table in BigQuery
    table = client.create_table(table)
    
    print(f"Table {table.table_id} created successfully.")

    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(data, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Success
    print(f"Data has been successfully retrieved, saved, and appended to the BigQuery table for {link}.")

    # Remove Temporary File
    os.remove('naivas_metadata.csv')

# Close the Selenium WebDriver
driver.quit()

#### **2. Fresh Food**

In [ ]:
# Set up Selenium Firefox WebDriver
options = Options()
options.headless = True  # Run Firefox in headless mode, without opening a browser window
driver = webdriver.Firefox(options=options)

# Define Links
links = [
        'naivas-bakery',
        'fruit-veggie',
        'dairy',
        'pre-packed-meat-products',
        'cold-deli',
        'naivas-butchery',
        'outsourced-bakery'
    ]

# Define Loop
for link in links:
    # Specify Website URL
    url = 'https://naivas.online/' + str(link)

    # Start the timer
    start_time = time.time()

    # Load Webpage
    driver.get(url)

    # Wait for Page to fully render
    driver.implicitly_wait(4)

    # Get Initial Page Height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll Down to Page Bottom
    while True:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  # Wait 4 seconds for page to load more content

        # Compare Final Scroll Height with Initial Scroll Height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # If Height is constant, exit loop
        last_height = new_height

        # Limit Number of Listings
        if len(BeautifulSoup(driver.page_source, 'html.parser').find_all('div', class_='h-full')) >= 500:
            break

    # Get Page Source after JavaScript Rendering
    page_source = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'lxml')
    groceries = soup.find_all('div', class_='h-full')

    # Extract grocery Listings from Jiji
    bigdata = pd.DataFrame()
    for grocery in groceries:
        try:
            product_name = grocery.find('span',class_='line-clamp-2 text-ellipsis').text.strip()
            product_category = 'Fresh Food'
            product_subcategory = link
            try:
                product_availability = grocery.find('p',class_='text-naivas-red text-sm font-bold').text.strip()
            except Exception as e:
                product_availability = np.nan
            try:
                final_price = grocery.find('span',class_='font-bold text-naivas-green mb-1 md:mb-0 pe-2').text.strip()
            except Exception as e:
                final_price = np.nan
            last_scraped = datetime.now()
            try:
                initial_price = grocery.find('span',class_='text-red-600 text-xs line-through font-light').text.strip()
            except Exception as e:
                initial_price = np.nan

            # Append Data to DataFrame
            data = pd.DataFrame({
                    'product_name':[product_name],
                    'product_category':[product_category],
                    'product_subcategory':[product_subcategory],
                    'product_availability':[product_availability],
                    'initial_price':[initial_price],
                    'final_price':[final_price],
                    'last_scraped':[last_scraped]
                })

            bigdata = pd.concat([bigdata,data])

        except Exception as e:
            pass

    # Handle Database Import Error
    bigdata.to_csv('naivas_metadata.csv',index=False)
    bigdata = pd.read_csv('naivas_metadata.csv')

    # Drop Duplicate Records
    bigdata['last_scraped'] = pd.to_datetime(bigdata['last_scraped'])


    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(bigdata, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
    sql = (
           'SELECT *'
           'FROM `cronjobs.naivas`'
           )
    
    # Run SQL Query
    data = client.query(sql).to_dataframe()

    # Delete Original Table
    client.delete_table(table_id)
    print(f"Table deleted successfully.")

    # Check Total Number of Duplicate Records
    duplicated = data.duplicated(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price']).sum()
    
    # Remove Duplicate Records
    data.drop_duplicates(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price'],inplace=True)

    # Define the dataset ID and table ID
    dataset_id = 'cronjobs'
    table_id = 'naivas'
    
    # Define the table schema
    schema = [
        bigquery.SchemaField("product_name", "STRING"),
        bigquery.SchemaField("product_category", "STRING"),
        bigquery.SchemaField("product_subcategory", "STRING"),
        bigquery.SchemaField("product_availability", "STRING"),
        bigquery.SchemaField("initial_price", "STRING"),
        bigquery.SchemaField("final_price", "STRING"),
        bigquery.SchemaField("last_scraped", "TIMESTAMP")
    ]
    
    # Define the table reference
    table_ref = client.dataset(dataset_id).table(table_id)
    
    # Create the table object
    table = bigquery.Table(table_ref, schema=schema)
    
    # Create the table in BigQuery
    table = client.create_table(table)
    
    print(f"Table {table.table_id} created successfully.")

    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(data, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Success
    print(f"Data has been successfully retrieved, saved, and appended to the BigQuery table for {link}")

    # Remove Temporary File
    os.remove('naivas_metadata.csv')

# Close the Selenium WebDriver
driver.quit()

#### **3. Baby & Kids**

In [ ]:
# Set up Selenium Firefox WebDriver
options = Options()
options.headless = True  # Run Firefox in headless mode, without opening a browser window
driver = webdriver.Firefox(options=options)

# Define Links
links = [
        'nursing-feeding',
        'baby-hygiene',
        'baby-skincare'
    ]

# Define Loop
for link in links:
    # Specify Website URL
    url = 'https://naivas.online/' + str(link)

    # Start the timer
    start_time = time.time()

    # Load Webpage
    driver.get(url)

    # Wait for Page to fully render
    driver.implicitly_wait(4)

    # Get Initial Page Height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll Down to Page Bottom
    while True:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  # Wait 4 seconds for page to load more content

        # Compare Final Scroll Height with Initial Scroll Height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # If Height is constant, exit loop
        last_height = new_height

        # Limit Number of Listings
        if len(BeautifulSoup(driver.page_source, 'html.parser').find_all('div', class_='h-full')) >= 500:
            break

    # Get Page Source after JavaScript Rendering
    page_source = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'lxml')
    groceries = soup.find_all('div', class_='h-full')

    # Extract grocery Listings from Jiji
    bigdata = pd.DataFrame()
    for grocery in groceries:
        try:
            product_name = grocery.find('span',class_='line-clamp-2 text-ellipsis').text.strip()
            product_category = 'Baby & Kids'
            product_subcategory = link
            try:
                product_availability = grocery.find('p',class_='text-naivas-red text-sm font-bold').text.strip()
            except Exception as e:
                product_availability = np.nan
            try:
                final_price = grocery.find('span',class_='font-bold text-naivas-green mb-1 md:mb-0 pe-2').text.strip()
            except Exception as e:
                final_price = np.nan
            last_scraped = datetime.now()
            try:
                initial_price = grocery.find('span',class_='text-red-600 text-xs line-through font-light').text.strip()
            except Exception as e:
                initial_price = np.nan

            # Append Data to DataFrame
            data = pd.DataFrame({
                    'product_name':[product_name],
                    'product_category':[product_category],
                    'product_subcategory':[product_subcategory],
                    'product_availability':[product_availability],
                    'initial_price':[initial_price],
                    'final_price':[final_price],
                    'last_scraped':[last_scraped]
                })

            bigdata = pd.concat([bigdata,data])

        except Exception as e:
            pass

    # Handle Database Import Error
    bigdata.to_csv('naivas_metadata.csv',index=False)
    bigdata = pd.read_csv('naivas_metadata.csv')

    # Drop Duplicate Records
    bigdata['last_scraped'] = pd.to_datetime(bigdata['last_scraped'])


    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(bigdata, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
    sql = (
           'SELECT *'
           'FROM `cronjobs.naivas`'
           )
    
    # Run SQL Query
    data = client.query(sql).to_dataframe()

    # Delete Original Table
    client.delete_table(table_id)
    print(f"Table deleted successfully.")

    # Check Total Number of Duplicate Records
    duplicated = data.duplicated(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price']).sum()
    
    # Remove Duplicate Records
    data.drop_duplicates(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price'],inplace=True)

    # Define the dataset ID and table ID
    dataset_id = 'cronjobs'
    table_id = 'naivas'
    
    # Define the table schema
    schema = [
        bigquery.SchemaField("product_name", "STRING"),
        bigquery.SchemaField("product_category", "STRING"),
        bigquery.SchemaField("product_subcategory", "STRING"),
        bigquery.SchemaField("product_availability", "STRING"),
        bigquery.SchemaField("initial_price", "STRING"),
        bigquery.SchemaField("final_price", "STRING"),
        bigquery.SchemaField("last_scraped", "TIMESTAMP")
    ]
    
    # Define the table reference
    table_ref = client.dataset(dataset_id).table(table_id)
    
    # Create the table object
    table = bigquery.Table(table_ref, schema=schema)
    
    # Create the table in BigQuery
    table = client.create_table(table)
    
    print(f"Table {table.table_id} created successfully.")

    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(data, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Success
    print(f"Data has been successfully retrieved, saved, and appended to the BigQuery table for {link}")

    # Remove Temporary File
    os.remove('naivas_metadata.csv')

# Close the Selenium WebDriver
driver.quit()

#### **4. Electronics**

In [ ]:
# Set up Selenium Firefox WebDriver
options = Options()
options.headless = True  # Run Firefox in headless mode, without opening a browser window
driver = webdriver.Firefox(options=options)

# Define Links
links = [
        'kitchen-appliances',
        'garment-care',
        'fridges-freezers',
        'cookers',
        'air-conditioning',
        'sound-system',
        'televisions',
        'washing-machine'
    ]

# Define Loop
for link in links:
    # Specify Website URL
    url = 'https://naivas.online/' + str(link)

    # Start the timer
    start_time = time.time()

    # Load Webpage
    driver.get(url)

    # Wait for Page to fully render
    driver.implicitly_wait(4)

    # Get Initial Page Height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll Down to Page Bottom
    while True:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  # Wait 4 seconds for page to load more content

        # Compare Final Scroll Height with Initial Scroll Height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # If Height is constant, exit loop
        last_height = new_height

        # Limit Number of Listings
        if len(BeautifulSoup(driver.page_source, 'html.parser').find_all('div', class_='h-full')) >= 500:
            break

    # Get Page Source after JavaScript Rendering
    page_source = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'lxml')
    groceries = soup.find_all('div', class_='h-full')

    # Extract grocery Listings from Jiji
    bigdata = pd.DataFrame()
    for grocery in groceries:
        try:
            product_name = grocery.find('span',class_='line-clamp-2 text-ellipsis').text.strip()
            product_category = 'Electronics'
            product_subcategory = link
            try:
                product_availability = grocery.find('p',class_='text-naivas-red text-sm font-bold').text.strip()
            except Exception as e:
                product_availability = np.nan
            try:
                final_price = grocery.find('span',class_='font-bold text-naivas-green mb-1 md:mb-0 pe-2').text.strip()
            except Exception as e:
                final_price = np.nan
            last_scraped = datetime.now()
            try:
                initial_price = grocery.find('span',class_='text-red-600 text-xs line-through font-light').text.strip()
            except Exception as e:
                initial_price = np.nan

            # Append Data to DataFrame
            data = pd.DataFrame({
                    'product_name':[product_name],
                    'product_category':[product_category],
                    'product_subcategory':[product_subcategory],
                    'product_availability':[product_availability],
                    'initial_price':[initial_price],
                    'final_price':[final_price],
                    'last_scraped':[last_scraped]
                })

            bigdata = pd.concat([bigdata,data])

        except Exception as e:
            pass

    # Handle Database Import Error
    bigdata.to_csv('naivas_metadata.csv',index=False)
    bigdata = pd.read_csv('naivas_metadata.csv')

    # Drop Duplicate Records
    bigdata['last_scraped'] = pd.to_datetime(bigdata['last_scraped'])


    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(bigdata, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
    sql = (
           'SELECT *'
           'FROM `cronjobs.naivas`'
           )
    
    # Run SQL Query
    data = client.query(sql).to_dataframe()

    # Delete Original Table
    client.delete_table(table_id)
    print(f"Table deleted successfully.")

    # Check Total Number of Duplicate Records
    duplicated = data.duplicated(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price']).sum()
    
    # Remove Duplicate Records
    data.drop_duplicates(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price'],inplace=True)

    # Define the dataset ID and table ID
    dataset_id = 'cronjobs'
    table_id = 'naivas'
    
    # Define the table schema
    schema = [
        bigquery.SchemaField("product_name", "STRING"),
        bigquery.SchemaField("product_category", "STRING"),
        bigquery.SchemaField("product_subcategory", "STRING"),
        bigquery.SchemaField("product_availability", "STRING"),
        bigquery.SchemaField("initial_price", "STRING"),
        bigquery.SchemaField("final_price", "STRING"),
        bigquery.SchemaField("last_scraped", "TIMESTAMP")
    ]
    
    # Define the table reference
    table_ref = client.dataset(dataset_id).table(table_id)
    
    # Create the table object
    table = bigquery.Table(table_ref, schema=schema)
    
    # Create the table in BigQuery
    table = client.create_table(table)
    
    print(f"Table {table.table_id} created successfully.")

    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(data, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Success
    print(f"Data has been successfully retrieved, saved, and appended to the BigQuery table for {link}")

    # Remove Temporary File
    os.remove('naivas_metadata.csv')

# Close the Selenium WebDriver
driver.quit()

#### **5. Cleaning**

In [ ]:
# Set up Selenium Firefox WebDriver
options = Options()
options.headless = True  # Run Firefox in headless mode, without opening a browser window
driver = webdriver.Firefox(options=options)

# Define Links
links = [
        'laundry',
        'dish-washing',
        'surface-cleaners-protection',
        'all-purpose-cleaners',
        'toilet-septic-care'
    ]

# Define Loop
for link in links:
    # Specify Website URL
    url = 'https://naivas.online/' + str(link)

    # Start the timer
    start_time = time.time()

    # Load Webpage
    driver.get(url)

    # Wait for Page to fully render
    driver.implicitly_wait(4)

    # Get Initial Page Height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll Down to Page Bottom
    while True:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  # Wait 4 seconds for page to load more content

        # Compare Final Scroll Height with Initial Scroll Height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # If Height is constant, exit loop
        last_height = new_height

        # Limit Number of Listings
        if len(BeautifulSoup(driver.page_source, 'html.parser').find_all('div', class_='h-full')) >= 500:
            break

    # Get Page Source after JavaScript Rendering
    page_source = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'lxml')
    groceries = soup.find_all('div', class_='h-full')

    # Extract grocery Listings from Jiji
    bigdata = pd.DataFrame()
    for grocery in groceries:
        try:
            product_name = grocery.find('span',class_='line-clamp-2 text-ellipsis').text.strip()
            product_category = 'Cleaning'
            product_subcategory = link
            try:
                product_availability = grocery.find('p',class_='text-naivas-red text-sm font-bold').text.strip()
            except Exception as e:
                product_availability = np.nan
            try:
                final_price = grocery.find('span',class_='font-bold text-naivas-green mb-1 md:mb-0 pe-2').text.strip()
            except Exception as e:
                final_price = np.nan
            last_scraped = datetime.now()
            try:
                initial_price = grocery.find('span',class_='text-red-600 text-xs line-through font-light').text.strip()
            except Exception as e:
                initial_price = np.nan

            # Append Data to DataFrame
            data = pd.DataFrame({
                    'product_name':[product_name],
                    'product_category':[product_category],
                    'product_subcategory':[product_subcategory],
                    'product_availability':[product_availability],
                    'initial_price':[initial_price],
                    'final_price':[final_price],
                    'last_scraped':[last_scraped]
                })

            bigdata = pd.concat([bigdata,data])

        except Exception as e:
            pass

    # Handle Database Import Error
    bigdata.to_csv('naivas_metadata.csv',index=False)
    bigdata = pd.read_csv('naivas_metadata.csv')

    # Drop Duplicate Records
    bigdata['last_scraped'] = pd.to_datetime(bigdata['last_scraped'])


    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(bigdata, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
    sql = (
           'SELECT *'
           'FROM `cronjobs.naivas`'
           )
    
    # Run SQL Query
    data = client.query(sql).to_dataframe()

    # Delete Original Table
    client.delete_table(table_id)
    print(f"Table deleted successfully.")

    # Check Total Number of Duplicate Records
    duplicated = data.duplicated(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price']).sum()
    
    # Remove Duplicate Records
    data.drop_duplicates(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price'],inplace=True)

    # Define the dataset ID and table ID
    dataset_id = 'cronjobs'
    table_id = 'naivas'
    
    # Define the table schema
    schema = [
        bigquery.SchemaField("product_name", "STRING"),
        bigquery.SchemaField("product_category", "STRING"),
        bigquery.SchemaField("product_subcategory", "STRING"),
        bigquery.SchemaField("product_availability", "STRING"),
        bigquery.SchemaField("initial_price", "STRING"),
        bigquery.SchemaField("final_price", "STRING"),
        bigquery.SchemaField("last_scraped", "TIMESTAMP")
    ]
    
    # Define the table reference
    table_ref = client.dataset(dataset_id).table(table_id)
    
    # Create the table object
    table = bigquery.Table(table_ref, schema=schema)
    
    # Create the table in BigQuery
    table = client.create_table(table)
    
    print(f"Table {table.table_id} created successfully.")

    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(data, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Success
    print(f"Data has been successfully retrieved, saved, and appended to the BigQuery table for {link}")

    # Remove Temporary File
    os.remove('naivas_metadata.csv')

# Close the Selenium WebDriver
driver.quit()

#### **6. Beauty & Cosmetics**

In [ ]:
# Set up Selenium Firefox WebDriver
options = Options()
options.headless = True  # Run Firefox in headless mode, without opening a browser window
driver = webdriver.Firefox(options=options)

# Define Links
links = [
        'health-wellness',
        'personal-care',
        'body-care',
        'fragrances',
        'haircare-styling',
        'face-care'
    ]

# Define Loop
for link in links:
    # Specify Website URL
    url = 'https://naivas.online/' + str(link)

    # Start the timer
    start_time = time.time()

    # Load Webpage
    driver.get(url)

    # Wait for Page to fully render
    driver.implicitly_wait(4)

    # Get Initial Page Height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll Down to Page Bottom
    while True:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  # Wait 4 seconds for page to load more content

        # Compare Final Scroll Height with Initial Scroll Height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # If Height is constant, exit loop
        last_height = new_height

        # Limit Number of Listings
        if len(BeautifulSoup(driver.page_source, 'html.parser').find_all('div', class_='h-full')) >= 500:
            break

    # Get Page Source after JavaScript Rendering
    page_source = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'lxml')
    groceries = soup.find_all('div', class_='h-full')

    # Extract grocery Listings from Jiji
    bigdata = pd.DataFrame()
    for grocery in groceries:
        try:
            product_name = grocery.find('span',class_='line-clamp-2 text-ellipsis').text.strip()
            product_category = 'Beauty & Cosmetics'
            product_subcategory = link
            try:
                product_availability = grocery.find('p',class_='text-naivas-red text-sm font-bold').text.strip()
            except Exception as e:
                product_availability = np.nan
            try:
                final_price = grocery.find('span',class_='font-bold text-naivas-green mb-1 md:mb-0 pe-2').text.strip()
            except Exception as e:
                final_price = np.nan
            last_scraped = datetime.now()
            try:
                initial_price = grocery.find('span',class_='text-red-600 text-xs line-through font-light').text.strip()
            except Exception as e:
                initial_price = np.nan

            # Append Data to DataFrame
            data = pd.DataFrame({
                    'product_name':[product_name],
                    'product_category':[product_category],
                    'product_subcategory':[product_subcategory],
                    'product_availability':[product_availability],
                    'initial_price':[initial_price],
                    'final_price':[final_price],
                    'last_scraped':[last_scraped]
                })

            bigdata = pd.concat([bigdata,data])

        except Exception as e:
            pass

    # Handle Database Import Error
    bigdata.to_csv('naivas_metadata.csv',index=False)
    bigdata = pd.read_csv('naivas_metadata.csv')

    # Drop Duplicate Records
    bigdata['last_scraped'] = pd.to_datetime(bigdata['last_scraped'])


    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(bigdata, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
    sql = (
           'SELECT *'
           'FROM `cronjobs.naivas`'
           )
    
    # Run SQL Query
    data = client.query(sql).to_dataframe()

    # Delete Original Table
    client.delete_table(table_id)
    print(f"Table deleted successfully.")

    # Check Total Number of Duplicate Records
    duplicated = data.duplicated(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price']).sum()
    
    # Remove Duplicate Records
    data.drop_duplicates(subset=['product_name', 'product_category', 'product_subcategory', 'product_availability', 'initial_price', 'final_price'],inplace=True)

    # Define the dataset ID and table ID
    dataset_id = 'cronjobs'
    table_id = 'naivas'
    
    # Define the table schema
    schema = [
        bigquery.SchemaField("product_name", "STRING"),
        bigquery.SchemaField("product_category", "STRING"),
        bigquery.SchemaField("product_subcategory", "STRING"),
        bigquery.SchemaField("product_availability", "STRING"),
        bigquery.SchemaField("initial_price", "STRING"),
        bigquery.SchemaField("final_price", "STRING"),
        bigquery.SchemaField("last_scraped", "TIMESTAMP")
    ]
    
    # Define the table reference
    table_ref = client.dataset(dataset_id).table(table_id)
    
    # Create the table object
    table = bigquery.Table(table_ref, schema=schema)
    
    # Create the table in BigQuery
    table = client.create_table(table)
    
    print(f"Table {table.table_id} created successfully.")

    # Define the BigQuery table ID
    table_id = 'project-adrian-julius-aluoch.cronjobs.naivas'

    # Load the data into the BigQuery table
    job = client.load_table_from_dataframe(data, table_id)

    # Wait for the job to complete
    while job.state != 'DONE':
        time.sleep(2)
        job.reload()
        print(job.state)

    # Success
    print(f"Data has been successfully retrieved, saved, and appended to the BigQuery table for {link}")

    # Remove Temporary File
    os.remove('naivas_metadata.csv')

# Close the Selenium WebDriver
driver.quit()